# Authentication

This notebook is how we authenticate our Neo4j database to TACC! We do this using tapis, and more specifically the pods API. 
Pods are a tool which allow for simple creation of a Neo4j DBMS hosted at TACC simply through some basic commands.
At a high level, we create a Tapis object associated with an individual account. Then, a pod is created under that username, with the Neo4j template. 
This pod has an associated username and password to connect to the DBMS. A simple command reveals this information. It is also straightforward to add in other users to have
admin access to this pod. The pod has an associated bolt url, which can be connected to via a Neo4j API like py2neo in a jupyter notebook. It can also be connected to via the 
Neo4j browser. And that's it! The Neo4j database is hosted and requires authentication to TACC via Tapis!

In [1]:
import os
import time
import json
import pprint as pp
import datetime
from getpass import getpass
import pytz

import pandas
import neo4jupyter
import py2neo
from py2neo import Graph, Node, Relationship, GraphService

from tapipy.tapis import Tapis

def show(res):
    try:
        pp.pprint(res.json())
    except:
        pp.pprint(res.text)

neo4jupyter.init_notebook_mode()

<IPython.core.display.Javascript object>

In [2]:
start = time.time()

# Base URL for Tapis
base_url = "https://icicle.tapis.io"
username = str(input("username"))

# Get Tapis object if it isn't already created.
try:
    if t.base_url == base_url and t.username == username and t.access_token:
        print("Tapis object already exists.")
        if t.access_token.expires_at < datetime.datetime.now(pytz.utc):
            print("Existing Tapis token expired, getting new token.")
            raise
    else:
        print("Creating new Tapis object.")
        raise
except:
    try:
        t = Tapis(base_url = base_url,
                  username = username,
                  password = getpass('password'))
        t.get_tokens()
    except Exception as e:
        print(f"\nBROKEN! timeout: {time.time() - start}\n")
        raise

# V3 Headers
header_dat = {"X-Tapis-token": t.access_token.access_token,
              "Content-Type": "application/json"}

# Service URL
url = f"{base_url}/v3"                   # remote

print(time.time() - start)
print(f"base_url: {base_url}")
print(f"serv_url: {url}")

10.01650619506836
base_url: https://icicle.tapis.io
serv_url: https://icicle.tapis.io/v3


In [3]:
# Returns all of the pods registered to the account. 
t.pods.get_pods()

[
 creation_ts: None
 data_attached: []
 data_requests: []
 description: testing pod function
 environment_variables: 
 
 pod_id: testrehs
 pod_template: neo4j
 roles_inherited: []
 roles_required: []
 status: COMPLETE
 status_container: 
 message: Pod phase in Succeeded, putting in COMPLETE status.
 phase: Succeeded
 start_time: 2022-07-29 20:34:15+00:00
 status_requested: ON
 update_ts: None
 url: testrehs.pods.icicle.tapis.io,
 
 creation_ts: None
 data_attached: []
 data_requests: []
 description: e
 environment_variables: 
 
 pod_id: testingcli
 pod_template: neo4j
 roles_inherited: []
 roles_required: []
 status: RUNNING
 status_container: 
 message: Pod is running.
 phase: Running
 start_time: 2022-10-04 03:43:49+00:00
 status_requested: ON
 update_ts: None
 url: testingcli.pods.icicle.tapis.io]

In [8]:
# Used to reference the pod
pod_id = str(input("Enter a pod ID")).lower()

In [5]:
# Use neo4j to create a pod for a neo4j DBMS
pod_template = str(input("enter a pod template (enter neo4j)"))

In [6]:
# Describe the pod
pod_description = str(input("enter a pod description"))

In [10]:
# Run this to create the pod
t.pods.create_pod(pod_id=pod_id, pod_template=pod_template, description=pod_description)

ServerDownError: message: conf.show_traceback = True; only for development:
 Traceback (most recent call last):
  File "/usr/local/lib/python3.10/site-packages/sqlalchemy/engine/base.py", line 1900, in _execute_context
    self.dialect.do_execute(
  File "/usr/local/lib/python3.10/site-packages/sqlalchemy/engine/default.py", line 736, in do_execute
    cursor.execute(statement, parameters)
psycopg2.errors.UniqueViolation: duplicate key value violates unique constraint "password_pkey"
DETAIL:  Key (pod_id)=(testingneo4jupyter) already exists.


The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/home/tapis/service/./req_utils.py", line 25, in error_handler
    raise exc
  File "/usr/local/lib/python3.10/site-packages/starlette/middleware/errors.py", line 162, in __call__
    await self.app(scope, receive, _send)
  File "/home/tapis/service/./req_utils.py", line 83, in __call__
    await self.app(scope, receive, send)
  File "

In [ ]:
# Setting permissions 
t.pods.set_pod_permission(pod_id=pod_id, user="jackkarp", level="ADMIN")
t.pods.set_pod_permission(pod_id=pod_id, user="mkyzubr", level="ADMIN")
t.pods.set_pod_permission(pod_id=pod_id, user="archita", level="ADMIN")
t.pods.set_pod_permission(pod_id=pod_id, user="tg882699", level="ADMIN")

In [11]:
# Setting username and password for authentication
username, password = t.pods.get_pod_credentials(pod_id=pod_id).user_username, t.pods.get_pod_credentials(pod_id=pod_id).user_password

In [18]:
username

'testingneo4jupyter'

In [24]:
password

'OWLDzASEUetNBIDezBGhVUppOupXZg'

In [23]:
pod_id

'testingneo4jupyter'

In [19]:
# Testing connection to the bolt service
graph = Graph(f"bolt+ssc://{pod_id}.pods.icicle.tapis.io:443", auth=(username, password), secure=True, verify=True)

In [20]:
graph

Graph('bolt+s://testingneo4jupyter.pods.icicle.tapis.io:443')

In [21]:
# Further testing connection via query
query = "create (:N)-[:R]->(:V)"
graph.run(query).to_data_frame()

""


In [22]:
# Test visualization of test nodes
neo4jupyter.draw(graph, {"Organization" : "alias"})